<font size="+3"><strong>8.4. Model Deployment</strong></font>

Ready for deployment! Over the last three lessons, we've built all the pieces we need for our application. We have a module for getting and storing our data. We have the code to train our model and clean its predictions. In this lesson, we're going to put all those pieces together and deploy our model with an API that others can use to train their own models and predict volatility. We'll start by creating a `model` for all the code we created in the last lesson. Then we'll complete our `main` module, which will hold our FastAPI application with two paths: one for model training and one for prediction. Let's jump in!

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests

from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository


c:\Users\Danjuma Ahmed H\miniconda3\Lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_directory" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


# Model Module

We created a lot of code in the last lesson to building, training, and making predictions with our GARCH(1,1) model. We want this code to be reusable, so let's put it in its own module.

Let's start by instantiating a repository that we'll use for testing our module as we build.

**Task 8.4.1:** Create a `SQLRepository` named `repo`. Be sure that it's attached to a SQLite connection.

- [Open a connection to a SQL database using sqlite3.](../%40textbook/10-databases-sql.ipynb#sqlite3)

In [2]:
connection = sqlite3.connect(settings.db_name, check_same_thread = False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


Now that we have the `repo` ready, we'll shift to our `model` module and build a `GarchModel` class to hold all our code from the last lesson.

**Task 8.4.2:** In the `model` module, create a definition for a `GarchModel` model class. For now, it should only have an `__init__` method. Use the docstring as a guide. When you're done, test your class using the assert statements below.

- [What's a class?](../%40textbook/21-python-object-oriented-programming.ipynb#Classes)
- [Write a class definition in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Defining-a-Class)
- [Write a class method in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Methods)
- [What's an assert statement?](../%40textbook/02-python-advanced.ipynb#Testing-Code)
- [Write an assert statement in Python.](../%40textbook/02-python-advanced.ipynb#Testing-Code)

In [3]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_ambuja = GarchModel(ticker="AMBUJACEM.BSE", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_ambuja.ticker == "AMBUJACEM.BSE"
assert gm_ambuja.repo == repo
assert not gm_ambuja.use_new_data
assert gm_ambuja.model_directory == settings.model_directory

**Task 8.4.3:** Turn your `wrangle_data` function from the last lesson into a method for your `GarchModel` class. When you're done, use the assert statements below to test the method by getting and wrangling data for the department store [Shoppers Stop](https://www.shoppersstop.com/).

- [What's a function?](../%40textbook/02-python-advanced.ipynb#Functions)
- [Write a function in Python.](../%40textbook/02-python-advanced.ipynb#Functions)
- [Write a class method in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Methods)
- [What's an assert statement?](../%40textbook/02-python-advanced.ipynb#Testing-Code)
- [Write an assert statement in Python.](../%40textbook/02-python-advanced.ipynb#Testing-Code)

In [4]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

date
2019-11-22   -1.815300
2019-11-25    0.440205
2019-11-26    2.556611
2019-11-27    0.897436
2019-11-28   -1.595369
Name: return, dtype: float64

**Task 8.4.4:** Using your code from the previous lesson, create a `fit` method for your `GarchModel` class. When you're done, use the code below to test it.

- [Write a class method in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Methods)
- [What's an assert statement?](../%40textbook/02-python-advanced.ipynb#Testing-Code)
- [Write an assert statement in Python.](../%40textbook/02-python-advanced.ipynb#Testing-Code)

In [5]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2417.65
Distribution:                  Normal   AIC:                           4843.30
Method:            Maximum Likelihood   BIC:                           4862.93
                                        No. Observations:                 1000
Date:                Thu, Nov 30 2023   Df Residuals:                      999
Time:                        14:59:57   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1619  7.583e-02      2.135  3.273e-02 [1.330e-02,  0.311]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.1520      0.244      0.624      0.533      [ -0.325,  0.629]
alpha[1]       0.0323  2.232e-02      1.445      0.148 [-1.150e-02,7.600e-02]
beta[1]        0.9496  5.027e-02     18.889  1.394e-79      [  0.851,  1.048]
=============================================================================

Covariance estimator: robust
"""

**Task 8.4.5:** Using your code from the previous lesson, create a `predict_volatility` method for your `GarchModel` class. Your method will need to return predictions as a dictionary, so you'll need to add your `clean_prediction` function as a helper method. When you're done, test your work using the assert statements below.

- [Write a class method in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Methods)
- [Write a function in Python.](../%40textbook/02-python-advanced.ipynb#Functions)
- [What's an assert statement?](../%40textbook/02-python-advanced.ipynb#Testing-Code)
- [Write an assert statement in Python.](../%40textbook/02-python-advanced.ipynb#Testing-Code)

In [6]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2023-11-30T00:00:00': 2.0734370196416987,
 '2023-12-01T00:00:00': 2.0911599432786834,
 '2023-12-04T00:00:00': 2.108415642660715,
 '2023-12-05T00:00:00': 2.125221241268814,
 '2023-12-06T00:00:00': 2.1415929303362318}

Things are looking good! There are two last methods that we need to add to our `GarchModel` so that we can save a trained model and then load it when we need it. When we learned about saving and loading files in Project 5, we used a context handler. This time, we'll streamline the process using the [joblib library](https://joblib.readthedocs.io/en/latest/). We'll also start writing our filepaths more programmatically using the [os library](https://docs.python.org/3/library/os.html).

**Task 8.4.6:** Create a `dump` method for your `GarchModel` class. It should save the model assigned to the `model` attribute to the folder specified in your configuration `settings`. Use the docstring as a guide, and then test your work below.

- [Write a class method in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Methods)
- [Save an object using joblib.](../%40textbook/02-python-advanced.ipynb#Saving-and-Loading-Files-with-joblib)
- [Create a file path using os.](../%40textbook/02-python-advanced.ipynb#Working-with-Filepaths)

In [7]:
timestamp = pd.Timestamp.now().isoformat()
ticker="SHOPERSTOP.BSE"
f"{timestamp}_{ticker}.pkl"

'2023-11-30T15:00:00.515291_SHOPERSTOP.BSE.pkl'

In [8]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

'models\\SHOPERSTOP.BSE.pkl'

**Task 8.4.7:** Create a `load` function below that will take a ticker symbol as input and return a model. When you're done, use the next cell to load the Shoppers Stop model you saved in the previous task.

- [Handle errors using `try` and `except` blocks in Python.](../%40textbook/02-python-advanced.ipynb#Error-Handling)
- [Create a file path using os.](../%40textbook/02-python-advanced.ipynb#Working-with-Filepaths)
- [Raise an `Exception` in Python.](../%40textbook/02-python-advanced.ipynb#Raising-Errors)

In [9]:
def load(ticker):

    """Load latest model from model directory.

    Parameters
    ----------
    ticker : str
        Ticker symbol for which model was trained.

    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = os.path.join(settings.model_directory, f"*{ticker}.pkl")

    # Try to find path of latest model
    try:
        model_path = sorted(glob(pattern))[-1]

    # Handle possible `IndexError`
    except IndexError:
        raise Exception(f"No model trained for filename '{ticker}'.")

    # Load model
    model = joblib.load(model_path)

    # Return model
    return model

In [10]:
# Assign load output to `model`
model_shop = load(ticker="SHOPERSTOP.BSE")

# Does function return an `ARCHModelResult`
assert isinstance(model_shop, ARCHModelResult)

# Check model parameters
model_shop.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2417.65
Distribution:                  Normal   AIC:                           4843.30
Method:            Maximum Likelihood   BIC:                           4862.93
                                        No. Observations:                 1000
Date:                Thu, Nov 30 2023   Df Residuals:                      999
Time:                        14:59:57   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1619  7.583e-02      2.135  3.273e-02 [1.330e-02,  0.311]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.1520      0.244      0.624      0.533      [ -0.325,  0.629]
alpha[1]       0.0323  2.232e-02      1.445      0.148 [-1.150e-02,7.600e-02]
beta[1]        0.9496  5.027e-02     18.889  1.394e-79      [  0.851,  1.048]
=============================================================================

Covariance estimator: robust
"""

**Task 8.4.8:** Transform your `load` function into a method for your `GarchModel` class. When you're done, test the method using the assert statements below.

- [Write a class method in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Methods)
- [What's an assert statement?](../%40textbook/02-python-advanced.ipynb#Testing-Code)
- [Write an assert statement in Python.](../%40textbook/02-python-advanced.ipynb#Testing-Code)

In [11]:
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Check that new `model_shop_test` doesn't have model attached
assert not hasattr(model_shop, "model")

# Load model
model_shop.load()

# Does `model_shop_test` have model attached?
assert hasattr(model_shop, "model")

model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2417.65
Distribution:                  Normal   AIC:                           4843.30
Method:            Maximum Likelihood   BIC:                           4862.93
                                        No. Observations:                 1000
Date:                Thu, Nov 30 2023   Df Residuals:                      999
Time:                        14:59:57   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1619  7.583e-02      2.135  3.273e-02 [1.330e-02,  0.311]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.1520      0.244      0.624      0.533      [ -0.325,  0.629]
alpha[1]       0.0323  2.232e-02      1.445      0.148 [-1.150e-02,7.600e-02]
beta[1]        0.9496  5.027e-02     18.889  1.394e-79      [  0.851,  1.048]
=============================================================================

Covariance estimator: robust
"""

Our `model` module is done! Now it's time to move on to the "main" course and add the final piece to our application.

# Main Module

Similar to the interactive applications we made in Projects 6 and 7, our first step here will be to create an `app` object. This time, instead of being a plotly application, it'll be a FastAPI application.

**Task 8.4.9:** In the `main` module, instantiate a FastAPI application named `app`.

- [Instantiate an application in FastAPI.](../%40textbook/22-apis.ipynb#Creating-a-Path)

In order for our `app` to work, we need to run it on a server. In this case, we'll run the server on our virtual machine using the [uvicorn library](https://www.uvicorn.org/).

**Task 8.4.10:** Go to the command line, navigate to the directory for this project, and start your app server by entering the following command.

```bash
uvicorn main:app --reload --workers 1 --host localhost --port 8008
```

Remember how the AlphaVantage API had a `"/query"` path that we accessed using a `get` HTTP request? We're going to build similar paths for our application. Let's start with an MVP example so we can learn how paths work in FastAPI. 

**Task 8.4.11:** Create a `"/hello"` path for your app that returns a greeting when it receives a `get` request.

- [Create an application path in FastAPI.](../%40textbook/22-apis.ipynb#Creating-a-Path)

We've got our path. Let's perform as `get` request to see if it works.

**Task 8.4.12:** Create a `get` request to hit the `"/hello"` path running at `"http://localhost:8008"`.

- [What's an HTTP request?](../%40textbook/22-apis.ipynb#RESTful-APIs)
- [Make an HTTP request using requests.](..//%40textbook/22-apis.ipynb#Making-an-HTTP-Request)

In [ ]:
url = "http://localhost:8008/hello"
response = requests.get(url=url)

print("response code:", response.status_code)
response.json()

## `"/fit"` Path

Our first path will allow the user to fit a model to stock data when they make a `post` request to our server. They'll have the choice to use new data from AlphaVantage, or older data that's already in our database. When a user makes a request, they'll receive a response telling them if the operation was successful or whether there was an error. 

One thing that's very important when building an API is making sure the user passes the correct parameters into the app. Otherwise, our app could crash! FastAPI works well with the [pydantic library](https://pydantic-docs.helpmanual.io/), which checks that each request has the correct parameters and data types. It does this by using special data classes that we need to define. Our `"/fit"` path will take user input and then output a response, so we need two classes: one for input and one for output.

**Task 8.4.13:** Create definitions for a `FitIn` and a `FitOut` data class. The `FitIn` class should inherit from the pydantic `BaseClass`, and the `FitOut` class should inherit from the `FitIn` class. Be sure to include type hints.

- [Write a class definition in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Defining-a-Class)
- [What's class inheritance?](../%40textbook/21-python-object-oriented-programming.ipynb#Inheritance)
- [What are type hints?](../%40textbook/22-apis.ipynb#Data-Classes-and-Type-Checking)
- [Define a data model in pydantic.](../%40textbook/22-apis.ipynb#Data-Classes-and-Type-Checking)

With our data classes defined, let's see how pydantic ensures our that users are supplying the correct input and our application is returning the correct output.

**Task 8.4.14:** Use the code below to experiment with your `FitIn` and `FitOut` classes. Under what circumstances does instantiating them throw errors? What class or classes are they instances of?

- [What's class inheritance?](../%40textbook/21-python-object-oriented-programming.ipynb#Inheritance)
- [What are type hints?](../%40textbook/22-apis.ipynb#Data-Classes-and-Type-Checking)
- [Define a data model in pydantic.](../%40textbook/22-apis.ipynb#Data-Classes-and-Type-Checking)

In [13]:
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker = "SHOPERSTOP.BSE",
    use_new_data = True,
    n_observations = 2000,
    p = 1,
    q = 1
)
print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = FitOut(
    ticker = "SHOPERSTOP.BSE",
    use_new_data = True,
    n_observations = 2000,
    p = 1,
    q = 1,
    success = True,
    message = "Great Job!!!"
)
print(fo)

ticker='SHOPERSTOP.BSE' use_new_data=True n_observations=2000 p=1 q=1
ticker='SHOPERSTOP.BSE' use_new_data=True n_observations=2000 p=1 q=1 success=True message='Great Job!!!'


In [14]:
dir(fi)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_a

In [15]:
fi.schema()

C:\Users\Danjuma Ahmed H\AppData\Local\Temp\ipykernel_13588\840422529.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  fi.schema()


{'properties': {'ticker': {'title': 'Ticker', 'type': 'string'},
  'use_new_data': {'title': 'Use New Data', 'type': 'boolean'},
  'n_observations': {'title': 'N Observations', 'type': 'integer'},
  'p': {'title': 'P', 'type': 'integer'},
  'q': {'title': 'Q', 'type': 'integer'}},
 'required': ['ticker', 'use_new_data', 'n_observations', 'p', 'q'],
 'title': 'FitIn',
 'type': 'object'}

One cool feature of FastAPI is that it can work in asynchronous scenarios. That's not something we need to learn for this project, but it does mean that we need to instantiate a `GarchModel` object each time a user makes a request. To make the coding easier for us, let's make a function to handle that process for us. 

**Task 8.4.15:** Create a `build_model` function in your `main` module. Use the docstring as a guide, and test your function below.

- [What's a function?](../%40textbook/02-python-advanced.ipynb#Functions)
- [Write a function in Python.](../%40textbook/02-python-advanced.ipynb#Functions)
- [What's an assert statement?](../%40textbook/02-python-advanced.ipynb#Testing-Code)
- [Write an assert statement in Python.](../%40textbook/02-python-advanced.ipynb#Testing-Code)

In [16]:
from main import build_model

# Instantiate `GarchModel` with function
model_shop = build_model(ticker="SHOPERSTOP.BSE", use_new_data=False)

# Is `SQLRepository` attached to `model_shop`?
assert isinstance(model_shop.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_shop.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_shop.ticker == "SHOPERSTOP.BSE"

# Is `use_new_data` attribute correct?
assert not model_shop.use_new_data

model_shop

We've got data classes, we've got a `build_model` function, and all that's left is to build the `"/fit"` path. We'll use our `"/hello"` path as a template, but we'll need to include more features, like error handling. 

**Task 8.4.16:** Create a `"/fit"` path for your `app`. It will take a `FitIn` object as input, and then build a `GarchModel` using the `build_model` function. The model will wrangle the needed data, fit to the data, and save the completed model. Finally, it will send a response in the form of a `FitOut` object. Be sure to handle any errors that may arise. 

- [Create an application path in FastAPI.](../%40textbook/22-apis.ipynb#Creating-a-Path)

Last step! Let's make a `post` request and see how our app responds.

**Task 8.4.17:** Create a `post` request to hit the `"/fit"` path running at `"http://localhost:8008"`. You should train a GARCH(1,1) model on 2000 observations of the Shoppers Stop data you already downloaded. Pass in your parameters as a dictionary using the `json` argument.

- [What's an argument?](../%40textbook/01-python-getting-started.ipynb#JSON)
- [What's an HTTP request?](../%40textbook/22-apis.ipynb#RESTful-APIs)
- [Make an HTTP request using requests.](..//%40textbook/22-apis.ipynb#Making-an-HTTP-Request)

In [ ]:
# URL of `/fit` path
url = "http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker" : "SHOPERSTOP.BSE",
    "use_new_data" : True,
    "n_observations" : 2000,
    "p": 1,
    "q": 1
}
# Response of post request
response = requests.post(url=url, json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

## `"/predict"` Path

For our `"/predict"` path, users will be able to make a `post` request with the ticker symbol they want a prediction for and the number of days they want to forecast into the future. Our app will return a forecast or, if there's an error, a message explaining the problem.

The setup will be very similar to our `"/fit"` path. We'll start with data classes for the in- and output.

**Task 8.4.18:** Create definitions for a `PredictIn` and `PredictOut` data class. The `PredictIn` class should inherit from the pydantic `BaseModel`, and the `PredictOut` class should inherit from the `PredictIn` class. Be sure to include type hints. The use the code below to test your classes.

- [Write a class definition in Python.](../%40textbook/21-python-object-oriented-programming.ipynb#Defining-a-Class)
- [What's class inheritance?](../%40textbook/21-python-object-oriented-programming.ipynb#Inheritance)
- [What are type hints?](../%40textbook/22-apis.ipynb#Data-Classes-and-Type-Checking)
- [Define a data model in pydantic.](../%40textbook/22-apis.ipynb#Data-Classes-and-Type-Checking)

In [18]:
from main import PredictIn, PredictOut

pi = PredictIn(ticker="SHOPERSTOP.BSE", n_days=5)
print(pi)

po = PredictOut(
    ticker="SHOPERSTOP.BSE", n_days=5, success=True, forecast={}, message="success"
)
print(po)

ticker='SHOPERSTOP.BSE' n_days=5
ticker='SHOPERSTOP.BSE' n_days=5 success=True forecast={} message='success'


Up next, let's create the path. The good news is that we'll be able to reuse our `build_model` function.

**Task 8.4.19:** Create a `"/predict"` path for your `app`. It will take a `PredictIn` object as input, build a `GarchModel`, load the most recent trained model for the given ticker, and generate a dictionary of predictions. It will then return a `PredictOut` object with the predictions included. Be sure to handle any errors that may arise.

- [Create an application path in FastAPI.](../%40textbook/22-apis.ipynb#Creating-a-Path)

Last step, let's see what happens when we make a `post` request...

**Task 8.4.20:** Create a `post` request to hit the `"/predict"` path running at `"http://localhost:8008"`. You should get the 5-day volatility forecast for Shoppers Stop. When you're satisfied, submit your work to the grader.

- [What's an HTTP request?](../%40textbook/22-apis.ipynb#RESTful-APIs)
- [Make an HTTP request using requests.](..//%40textbook/22-apis.ipynb#Making-an-HTTP-Request)

In [19]:
# URL of `/predict` path
url = "http://localhost:8008/predict"
# Data to send to path
json = {
    "ticker" : "SHOPERSTOP.BSE",
    "n_days": 5
}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

{'ticker': 'SHOPERSTOP.BSE',
 'n_days': 5,
 'success': True,
 'forecast': {'2023-04-14T00:00:00': 1.8691015002138183,
  '2023-04-17T00:00:00': 1.8860637972563472,
  '2023-04-18T00:00:00': 1.9025905613644554,
  '2023-04-19T00:00:00': 1.9186978369762901,
  '2023-04-20T00:00:00': 1.9344007734310043},
 'message': ''}

We did it! Better said, **you** did it. You got data from the AlphaVantage API, you stored it in a SQL database, you built and trained a GARCH model to predict volatility, and you created your own API to serve predictions from your model. That's data engineering, data science, and model deployment all in one project. If you haven't already, now's a good time to give yourself a pat on the back. You definitely deserve it.